# Data simulation: how does it work?
The bias $\alpha$ and precision $\beta$ are reflected in the data as follows. Take the following example data for a subject:

|x|-40|-30|-20|-10|0|10|20|30|40|
|-|---|---|---|---|-|--|--|--|--|
|n|2|2|2|2|2|2|2|2|2|
|r|0|0|0|1|1|1|2|2|2|

In this case, we can see there is an *interval of uncertainty*, where the subject does not always answer 'faster' or 'slower'. Furthermore, this interval is centered around 0. If we were to decrease the precision $\beta$, this would mean that the interval of uncertainty would become larger, i.e.:

|x|-40|-30|-20|-10|0|10|20|30|40|
|-|---|---|---|---|-|--|--|--|--|
|n|2|2|2|2|2|2|2|2|2|
|r|0|1|1|1|1|1|1|1|2|

Changing the bias would mean changing the center of the uncertainty interval, i.e.:

|x|-40|-30|-20|-10|0|10|20|30|40|
|-|---|---|---|---|-|--|--|--|--|
|n|2|2|2|2|2|2|2|2|2|
|r|0|1|1|1|2|2|2|2|2|

### Installating modules, specifying functions, etc.

In [2]:
%%capture
!pip install pymc3

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns
from pymc3 import math
from scipy.stats import norm

sns.set_context('talk')

The following module was added (compared to the original code) to enable random sampling:

In [ ]:
import random as random

In [ ]:
def cumulative_normal(x, alpha, beta):
    # Cumulative distribution function for the standard normal distribution
    return 0.5 + 0.5 * math.erf((x - alpha) / (beta * np.sqrt(2)))

Set a seed for the random number generator to help with reproducibility:

In [ ]:
random.seed(42)

## Functions

The following function allows us to output both statistics for any subject:

In [ ]:
def bayes_fit(name, x, n, r, stats = True):
  results = [name]
  
  with pm.Model():

    alpha = pm.Uniform("alpha", lower=-40.5, upper=40.5)
    beta = pm.HalfNormal("beta", 10)

    theta = pm.Deterministic(
        "theta", cumulative_normal(x, alpha, beta)
    )

    r_ = pm.Binomial("rij", p=theta, n=n, observed=r)
    trace = pm.sample(
        chains=4, cores=4, tune=2000, draws=4000, return_inferencedata=True, target_accept=0.95 
        #note that the target acceptance has been increased to help with divergence problems
    )

  if stats == True:
    stats = az.summary(trace, ["alpha", "beta"])
    results.append(stats)

  return(results)

## Example: how do the data change according to the hypotheses?
The base of the example:

In [ ]:
x1 = [-40, -30, -20, -10, 0, 10, 20, 30, 40]
n1 = [2] * 9
r1 = [0, 0, 0, 1, 1, 1, 2, 2, 2]

fit1 = bayes_fit('fit1', x = x1, n = n1, r = r1, stats = True, plot = False)
fit1[1]

ERROR:pymc3:There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.043,6.285,-11.877,11.954,0.073,0.063,7629.0,6746.0,1.0
beta,14.015,4.501,6.613,22.507,0.052,0.037,7050.0,7061.0,1.0


Decreasing the precision (increasing $\beta$):

In [ ]:
x2 = [-40, -30, -20, -10, 0, 10, 20, 30, 40]
n2 = [2] * 9
r2 = [0, 1, 1, 1, 1, 1, 1, 1, 2]

fit2 = bayes_fit('fit2', x = x2, n = n2, r = r2, stats = True, plot = False)
fit2[1]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.205,8.578,-15.941,16.553,0.095,0.087,8444.0,7120.0,1.0
beta,25.794,5.097,16.706,35.387,0.056,0.039,8302.0,7950.0,1.0


Increasing the bias $\alpha$:

In [ ]:
x3 = [-40, -30, -20, -10, 0, 10, 20, 30, 40]
n3 = [2] * 9
r3 = [0, 1, 1, 1, 2, 2, 2, 2, 2]

fit3 = bayes_fit('fit3', x = x3, n = n3, r = r3, stats = True, plot = False)
fit3[1]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-20.558,6.368,-32.668,-8.382,0.081,0.064,6424.0,3894.0,1.0
beta,14.376,4.761,6.246,23.245,0.065,0.046,5163.0,6033.0,1.0


## Hypothesis
The hypothesis is that the bias ($\alpha$) and precision ($\beta$) will be reduced when the drug dose is increased.

To simulate the data corresponding to this hypothesis, we take the following steps:
- Retrieve the data for each subject separately
- Split the data into the exteroceptive and interoceptive conditions
- Repeat 3 times with varying intensity: 
  * Increase the uncertainty interval and decrease the bias in the data

Some things to note about the original data:
- The frequencies -30, 20, -10, 10, 20, and 30 are each sampled 2 times per participant (for both exteroception and interoception).
- When the uncertainty zone increases, this will influence the sampled frequencies. We approximate this effect.

### Single subject
We first demonstrate the concept for 1 subject.

1. Read in the data

In [ ]:
psychophysics_df = pd.read_csv('https://github.com/embodied-computation-group/CardioceptionPaper/raw/main/data/Del2_merged.txt')

2. Retrieve the interoceptive condition of the first subject only (subject # 19)

In [ ]:
this_df = psychophysics_df[(psychophysics_df.Modality == 'Intero') & (psychophysics_df.Subject == 'sub_0019')]

3. Retrieve the `Alpha` (intensity, later `x`) and `Decision` ('more' / 'less') variables

In [ ]:
this_df = this_df[['Alpha', 'Decision']]

4. Create the `x`, `n` and `r` variables of interest

In [ ]:
x, n, r = np.zeros(163), np.zeros(163), np.zeros(163)

for ii, intensity in enumerate(np.arange(-40.5, 41, 0.5)):
    x[ii] = intensity
    n[ii] = sum(this_df.Alpha == intensity)
    r[ii] = sum((this_df.Alpha == intensity) & (this_df.Decision == "More"))

# remove no responses trials
validmask = n != 0
xij, nij, rij = x[validmask], n[validmask], r[validmask]

5. Determine the uncertainty interval

In [ ]:
interval_min = min(xij[np.where((rij / nij != 0) & (rij / nij != 1))])
interval_max = max(xij[np.where((rij / nij != 0) & (rij / nij != 1))])

print(interval_min, interval_max)

-1.5 14.5


6. Set the change levels

Note that these can be adapted freely, their levels are not based on anything at the moment.

In [ ]:
change5 = 0.1
change10 = 0.2
change15 = 0.3

7. Calculate the old and new width and bias of the uncertainty interval

In [ ]:
width = abs(interval_min) + abs(interval_max)
bias = (interval_min + interval_max) / 2

# for the 5 mg dose
width5 = width * (1 + change5) # the width increases with change5 %
bias5 = bias * (1 - change5) # the bias decreases with change5 %
uncertainty5_min = bias5 - (width5 / 2) # the new minimum
uncertainty5_max = bias5 + (width5 / 2) # the new maximum

# for the 10 mg dose
width10 = width * (1 + change10) # the width increases with change10 %
bias10 = bias * (1 - change10) # the bias decreases with change10 %
uncertainty10_min = bias10 - (width10 / 2)
uncertainty10_max = bias10 + (width10 / 2)

# for the 15 mg dose
width15 = width * (1 + change15) # the width increases with change10 %
bias15 = bias * (1 - change15) # the bias decreases with change10 %
uncertainty15_min = bias15 - (width15 / 2)
uncertainty15_max = bias15 + (width15 / 2)

8. Adapt the sampling frequencies for each dose

Each subject undergoes 120 trials for each dose. Within those 120, 60 are for the interoceptive condition. Out of the 60, 12 are at set frequencies (-30, -20, -10, 10, 20, 30). There are therefore 48 intensities left to be sampled freely. The data shows that each participant has at least 23 unique frequencies 
and at most 40. 6 of those are the set frequencies, so there are 17 to 34 unique intensities left to be sampled freely. 

The sampling distribution is approximated by a normal distribution with mean the bias (sampling is equally distributed among both sides of the mean) and standard deviation $(0.8 * \text{width of uncertainty interval})/2$. Note that the standard deviation is not based on any reasoning per sé, but was determined through experimentation (it was varied until the resulting data was in correspondence with the desired effects). As a result of this setting, most samplling will fall within the uncertainty region which is in correspondence with the expectations for the real life experiments.

In [ ]:
# for the 5 mg dose
newdat5x = [-30, -20, -10, 10, 20, 30]
newdat5n = [2] * 6 # new data with set intensities each sampled twice

newx = [0] * 48 # space for the new x values

# sample between 18 and 34 unique intensities for the new data
numx = random.choice(range(17,34)) # number of unique intensities
newx = np.random.normal(bias5, (0.8 * uncertainty5_max) / 0.5, numx)
newx = np.around(np.ndarray.tolist(newx)) * 0.5

for i in range(0, len(newx)):
  if (newx[i] - int(newx[i])) == 0:
    newx[i] = newx[i] + random.choice([-0.5, 0.5])

newn = [1] * len(newx)
while sum(newn) < 48:
  ind = random.choice(range(0, len(newx)))
  newn[ind] = newn[ind] + 1

newdat5x = newdat5x + newx.tolist()
newdat5n = newdat5n + newn

newdat5r = [0] * len(newdat5x)

# then use this to generate the values for r
for i in range(0, len(newdat5x)):
  if newdat5x[i] < uncertainty5_min:
    newdat5r[i] = 0
  elif newdat5x[i] > uncertainty5_max:
    newdat5r[i] = newdat5n[i]
  else:
    newdat5r[i] =  round(norm.cdf(newdat5x[i], bias5, 0.95 * (width5 / 2)) * newdat5n[i])

9. Calculate the new bias and precision

In [ ]:
fit5 = bayes_fit(name = 'new dose 5 mg, subject 19', x = newdat5x, n = newdat5n, r = newdat5r, stats = True)
fit5[1]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,4.964,1.810,1.617,8.396,0.020,0.015,8397.0,7448.0,1.0
beta,7.659,1.957,4.504,11.364,0.022,0.016,8135.0,8638.0,1.0


In [ ]:
fit = bayes_fit(name = 'original dose 0 mg, subject 19', x = xij, n = nij, r = rij, stats = True, plot = False)
fit[1]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,7.309,1.741,4.016,10.533,0.019,0.014,8333.0,7782.0,1.0
beta,7.890,1.934,4.655,11.390,0.022,0.016,9036.0,7749.0,1.0


The same principles can be applied to the exteroceptive data. In that case, we expect that the bias does not change much because it already is very low. However, the precision is expected to change in a similar way to the interoceptive condition (as illustrated above).